# MultibodyPlant로 PyPlot Animation 튜터리얼(PyPlot Animation with MultibodyPlant Tutorial)
notebook 실행 방법은 [index](./index.ipynb)를 참조하자.


## Selecting matplotlib Backends

Jupyter notebook은 다양한 백엔드를 선택할 수 있는 `%matplotlib` 를 제공합니다:
https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-matplotlib

일반적으로 선택할 수 있는 backends는 비대화적인 방식 (`inline`)과 대화적인 방식 (`notebook`, `tk`) 두 가지 종류가 있다. 이 notebook에서는 몇 가지 옵션을 보여드리며 주석 해제/설정을 통해 직접 시험해볼 수 있다.

**주의: Deepnote와 Google Colab는 `inline` backend만 사용하는 것을 권장한다.**

먼저, 사용 가능한 옵션을 살펴보겠다. 일부 백엔드는 필요한 의존성이 모두 설치되어 있지 않아 작동하지 않을 수 있다.

In [ ]:
%matplotlib --list

이제 backends 중에 하나를 선택하자. 아래 옵션은 대부분의 시스템에서 실행 가능할 확률이 높다.

커널 세션 중 백엔드는 한 번만 변경할 수 있다. 백엔드를 다시 변경하려면 세션을 재시작해야 한다.

In [ ]:
# This is non-interactive: it shows static plots inline
%matplotlib inline

# This is interactive: it shows dynamic plots in the notebook
# %matplotlib notebook

# This is interactive: it shows dynamic plots in separate GUI windows
# %matplotlib tk

## Imports

In [ ]:
from IPython.display import HTML
from matplotlib import animation
import numpy as np

In [ ]:
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder
from pydrake.systems.planar_scenegraph_visualizer import (
    ConnectPlanarSceneGraphVisualizer)

## Pendulum 정의 예제(Define Pendulum Example)

This function is consolidated from `run_planar_scenegraph_visualizer.py`.

In [ ]:
def run_pendulum_example(duration=1., playback=True, show=True):
    """
    Runs a simulation of a pendulum.

    Arguments:
        duration: Simulation duration (sec).
        playback: Enable pyplot animations to be produced.
    """
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, 0.)
    parser = Parser(plant)
    parser.AddModels(
        url="package://drake/examples/pendulum/Pendulum.urdf")
    plant.Finalize()

    T_VW = np.array([[1., 0., 0., 0.],
                     [0., 0., 1., 0.],
                     [0., 0., 0., 1.]])
    visualizer = ConnectPlanarSceneGraphVisualizer(
        builder, scene_graph, T_VW=T_VW, xlim=[-1.2, 1.2],
        ylim=[-1.2, 1.2], show=show)
    if playback:
        visualizer.start_recording()

    diagram = builder.Build()
    simulator = Simulator(diagram)
    simulator.Initialize()
    simulator.set_target_realtime_rate(1.)

    # Fix the input port to zero.
    plant_context = diagram.GetMutableSubsystemContext(
        plant, simulator.get_mutable_context())
    plant.get_actuation_input_port().FixValue(
        plant_context, np.zeros(plant.num_actuators()))
    plant_context.SetContinuousState([0.5, 0.1])
    simulator.AdvanceTo(duration)

    if playback:
        visualizer.stop_recording()
        ani = visualizer.get_recording_as_animation()
        return ani
    else:
        return None

## Run without Playback

If you have a non-interactive backend, you will not see any animation.
Additionally, you will see a UserWarning that it is using a non-GUI backend.

If you have an interactive backend, you should see the simulation animation
as it happens.

If you select a GUI option, this will open a new figure each time you run the
following cell.

In [ ]:
run_pendulum_example(playback=False)

## Run with Playback

If you have a non-interactive backend, you will not see any animation
in the first output.

If you have an interactive backend, you will see animation in the first output
(as the simulation happens). Additionally, the direct animation plot itself will
loop its playback.

In [ ]:
ani = run_pendulum_example(playback=True)

Given that you recorded playback, now you can produce an animation (regardless
of your backend) either as:

* A JavaScript HTML widget - allows for slightly finer-grained control
* An HTML5 video - requires ffmpeg, which is not installed as part of Drake's
dependencies

In [ ]:
HTML(ani.to_jshtml())

In [ ]:
if animation.writers.is_available("ffmpeg"):
    display(HTML(ani.to_html5_video()))

If you do not want to render the image (only the animation), then pass `show=False` in to the constructor of `PlanarSceneGraphVisualizer()`.

In [ ]:
ani = run_pendulum_example(playback=True, show=False)
HTML(ani.to_jshtml())